Loading of all necessary packages

In [86]:
#import necessary packages
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import re
from bs4 import BeautifulSoup
from time import time
from time import sleep
from random import randint
from IPython.core.display import clear_output
from warnings import warn
from datetime import datetime
import csv
from tabulate import tabulate
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

This Notebook has the aim to scrape data from a job site and send it via email to someone

Steps:
    1. Extract (ID, Beschreibung, Datum, Link)
    2. Setup email and send

#    1. Extract (ID, Beschreibung, Datum, Link)

We start with the scraper, he will open the website and scrape the possible 10 pages of the job page.

In [87]:
#Set the variables for the scraper
headers = {"Accept-Language": "en-US, en;q=0.5"}

#We use 10 pages for our scraper
pages = [str(i) for i in range(1,10)]

In [88]:
# List to store data in
IDs = []
Dates = []
Titles = []
Links = []
Descriptions = []

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every page in the interval 1-5
for page in pages:

    # Make a get request
    response = get('https://www.dgpraec.de/jobboerse/page/' + page, headers = headers)

    # Pause the loop
    sleep(randint(8,15))

    # Monitor the requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Break the loop if the number of requests is greater than expected
    if requests > 72:
        warn('Number of requests was greater than expected.')  
        break 

    # Parse the content of the request with BeautifulSoup
    page_html = BeautifulSoup(response.text, 'html.parser')

    # Select all the Job containers from a single page
    job_container = page_html.find_all('article', class_ =lambda value: value and value.startswith("post-"))

    # For every job of these
    for container in job_container:
    
        # IDs
        ID = container.get('id')
        IDs.append(ID)
        
        #Dates
        Date = container.div.text
        Dates.append(str(Date))
        
        #Titles
        Title = container.a.text
        Titles.append(Title)
        
        #Links
        Link = container.find_all('a')
        Links.append(Link)
        
        #Descriptions
        Description = container.p.text
        Descriptions.append(Description)

Request:4; Frequency: 0.06518174916027566 requests/s


In [89]:
#Create dataframe and add the data
Job_list = pd.DataFrame({'ID': IDs, 
                        'Date': Dates,
                        'Title': Titles,
                        'Link': Links,
                        'Description': Descriptions})
print(Job_list.info())
Job_list

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 5 columns):
Date           38 non-null object
Description    38 non-null object
ID             38 non-null object
Link           38 non-null object
Title          38 non-null object
dtypes: object(5)
memory usage: 1.6+ KB
None


,Date,Description,ID,Link,Title
0,\n 15. März 2019,"Universitätsklinikum Leipzig, Bereich Plastisc...",post-6007,"[<a href=""https://www.dgpraec.de/jobs/facharzt...",Facharzt in Leipzig gesucht
1,\n 14. März 2019,"Die Klinik für Plastische Chirurgie, Hand- und...",post-5998,"[<a href=""https://www.dgpraec.de/jobs/oberaerz...",Oberärztin/-arzt in Hannover gesucht
2,\n 14. März 2019,Unsere renommierte Privatpraxis im Südwesten i...,post-5995,"[<a href=""https://www.dgpraec.de/jobs/fachaerz...",Fachärztin / Facharzt für Plastische & Ästheti...
3,\n 14. März 2019,Im Rahmen einer Stellennachbesetzung ist vorau...,post-5989,"[<a href=""https://www.dgpraec.de/jobs/assisten...",Assistenzarzt/-ärztin Handchirurgie oder Plast...
4,\n 13. März 2019,"Wir, die Alster-Klinik, sind eine Privatklinik...",post-5979,"[<a href=""https://www.dgpraec.de/jobs/facharzt...",Facharzt/in für Plastische und Ästhetische Chi...
5,\n 13. März 2019,Die Klinikum Ernst von Bergmann Klinik gGmbH s...,post-5968,"[<a href=""https://www.dgpraec.de/jobs/oberarzt...",Oberarzt / Oberärztin für die Abteilung der P...
6,\n 12. März 2019,"Für unsere Klinik für Plastische, Hand- und wi...",post-5946,"[<a href=""https://www.dgpraec.de/jobs/oberarzt...",Oberarzt (w/m) für Plastische und Ästhetische ...
7,\n 11. März 2019,"Für die Klinik für Plastische, Ästhetische und...",post-5941,"[<a href=""https://www.dgpraec.de/jobs/assisten...",Assistenzarzt (m/w/d) in Wetzlar gesucht
8,\n 7. März 2019,Wir bieten: bestens eingeführte Praxis seit üb...,post-5932,"[<a href=""https://www.dgpraec.de/jobs/praxissi...",Praxissitz in Gemeinschaftspraxis für Plastisc...
9,\n 5. März 2019,Das Alfried Krupp Krankenhaus gehört mit 900 B...,post-5928,"[<a href=""https://www.dgpraec.de/jobs/assisten...",Assistenzarzt (m/w/d) für die Plastische Chiru...


In [91]:
#Cleaning of the date
Job_list['Date'] = Job_list.Date.str.strip()
Job_list['Date'] = Job_list.Date.str.replace('Januar', 'January')
Job_list['Date'] = Job_list.Date.str.replace('Februar', 'February')
Job_list['Date'] = Job_list.Date.str.replace('März', 'March')
Job_list['Date'] = Job_list.Date.str.replace('April', 'April')
Job_list['Date'] = Job_list.Date.str.replace('Mai', 'May')
Job_list['Date'] = Job_list.Date.str.replace('Juni', 'June')
Job_list['Date'] = Job_list.Date.str.replace('Juli', 'July')
Job_list['Date'] = Job_list.Date.str.replace('August', 'August')
Job_list['Date'] = Job_list.Date.str.replace('September', 'September')
Job_list['Date'] = Job_list.Date.str.replace('Oktober', 'October')
Job_list['Date'] = Job_list.Date.str.replace('November', 'November')
Job_list['Date'] = Job_list.Date.str.replace('Dezember', 'December')
Job_list['Date'] = pd.to_datetime(Job_list.Date)
Job_list['Date']

0    2019-03-15
1    2019-03-14
2    2019-03-14
3    2019-03-14
4    2019-03-13
5    2019-03-13
6    2019-03-12
7    2019-03-11
8    2019-03-07
9    2019-03-05
10   2019-03-05
11   2019-03-04
12   2019-03-01
13   2019-03-01
14   2019-02-25
15   2019-02-19
16   2019-02-18
17   2019-02-18
18   2019-02-13
19   2019-02-12
20   2019-02-12
21   2019-02-11
22   2019-02-11
23   2019-02-07
24   2019-02-06
25   2019-01-30
26   2019-01-23
27   2019-01-22
28   2019-01-22
29   2019-01-17
30   2019-01-16
31   2019-01-16
32   2019-01-16
33   2019-01-15
34   2019-01-09
35   2019-01-07
36   2019-01-07
37   2018-11-22
Name: Date, dtype: datetime64[ns]

#    2. Setup email and send

We need an algorithm which goes as follows -if the script will be run daily-: If one of the ids in the new list will matched with the new result, then send email

In [98]:
#We load the old results into our script.
Job_list_old = pd.read_csv('Output_old.CSV', engine='python', encoding='utf-8', index_col=0)
print(Job_list_old.head(5))

         Date                                        Description         ID  \
0  2019-03-15  Universitätsklinikum Leipzig, Bereich Plastisc...  post-6007   
1  2019-03-14  Die Klinik für Plastische Chirurgie, Hand- und...  post-5998   
2  2019-03-14  Unsere renommierte Privatpraxis im Südwesten i...  post-5995   
3  2019-03-14  Im Rahmen einer Stellennachbesetzung ist vorau...  post-5989   
4  2019-03-13  Wir, die Alster-Klinik, sind eine Privatklinik...  post-5979   

                                                Link  \
0  [<a href="https://www.dgpraec.de/jobs/facharzt...   
1  [<a href="https://www.dgpraec.de/jobs/oberaerz...   
2  [<a href="https://www.dgpraec.de/jobs/fachaerz...   
3  [<a href="https://www.dgpraec.de/jobs/assisten...   
4  [<a href="https://www.dgpraec.de/jobs/facharzt...   

                                               Title  
0                        Facharzt in Leipzig gesucht  
1               Oberärztin/-arzt in Hannover gesucht  
2  Fachärztin / Facharz

In [99]:
#Every job has an ID. So our job is now to compare the current results (Job_list) with the old results (Job_list_old)
str(Job_list.ID) == str(Job_list_old.ID)
#False means that there is a new job in the list
#True means no new jobs

True

In [100]:
#Creation of the difference list for testing purposes
Job_list_difference = pd.concat([Job_list_old,Job_list])
Job_list_difference = Job_list_difference.reset_index()
Job_list_difference = Job_list_difference.drop_duplicates(subset = ['ID'], keep=False)
Job_list_difference = Job_list_difference.reset_index()
Job_list_difference = Job_list_difference.drop(Job_list_difference.columns[[0, 1, 3, 4]], axis=1)
Job_list_difference['Date'] = pd.to_datetime(Job_list_difference.Date).dt.strftime('%d-%m-%y')
Job_list_difference.sort_values(by=['Date'])
Job_list_difference

,Date,Link,Title


In [95]:
#Setting up of the email variables
me = 'sending email adress'
password = '***'
server = 'smtp.gmail.com:587'
you = 'receiver email adress'

#The text
text = """
Dear receiver,

This is the email to inform you about the new job(s):

{table}

Regards,

The sender"""

html = """
<html><body><p>Dear receiver,</p>
<p>This is the email to inform you about the new job(s)</p>
{table}
<p>Regards,</p>
<p>Me</p>
</body></html>
"""

In [96]:
#An algorithm could look like this

#Check if the IDs are the same
if str(Job_list.Title) == str(Job_list_old.Title):
    print("Same - no email")
else:
    print("not same - email triggered")
    
#Creation of difference list
    Job_list_difference = pd.concat([Job_list_old,Job_list])
    Job_list_difference = Job_list_difference.reset_index()
    Job_list_difference = Job_list_difference.drop_duplicates(subset = ['Title'], keep=False)
    Job_list_difference = Job_list_difference.reset_index()
    Job_list_difference = Job_list_difference.drop(Job_list_difference.columns[[0, 1, 3, 4]], axis=1)
    Job_list_difference['Date'] = pd.to_datetime(Job_list_difference.Date).dt.strftime('%d-%m-%y')
    Job_list_difference.sort_values(by=['Date'])
    Job_list_difference.to_csv('Output_difference.CSV',sep=',')
    
#Sending of email
    with open('Output_difference.CSV', encoding='utf-8') as input_file:
        reader = csv.reader(input_file)
        data = list(reader)
    
    text = text.format(table=tabulate(data, headers="firstrow", tablefmt="grid"))
    html = html.format(table=tabulate(data, headers="firstrow", tablefmt="html"))

    message = MIMEMultipart(
        "alternative", None, [MIMEText(text), MIMEText(html,'html')])

    message['Subject'] = "Your data"
    message['From'] = me
    message['To'] = you
    server = smtplib.SMTP(server)
    server.ehlo()
    server.starttls()
    server.login(me, password)
    server.sendmail(me, you, message.as_string())
    server.quit()

not same - email triggered


In [97]:
#Finally we need to save the "old" results into a csv
Job_list.to_csv('Output_old.CSV',sep=',')
#IF the script is running daily we then have a reference point for the current data